In [3]:
import tensorflow_datasets as tfds
import tensorflow as tf

import time
import numpy as np


In [4]:
print( tf.__version__)

2.3.0


In [1]:
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1LeRPXt815fFM_1jXNEIPich6D_Hf5oxB" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1LeRPXt815fFM_1jXNEIPich6D_Hf5oxB" -o src

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    657      0 --:--:-- --:--:-- --:--:--   657
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 6171k    0 6171k    0     0  6789k      0 --:--:-- --:--:-- --:--:-- 6789k


In [2]:
!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1PdOCa1tl1li4QKQYtdbaqqmghVJWmkFo" > /dev/null
!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1PdOCa1tl1li4QKQYtdbaqqmghVJWmkFo" -o target

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    731      0 --:--:-- --:--:-- --:--:--   731
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 5107k    0 5107k    0     0  6080k      0 --:--:-- --:--:-- --:--:-- 6080k


In [5]:
source = open('/content/src', 'r', encoding='utf-8')
target = open('/content/target', 'r', encoding='utf-8')

source_lines = source.readlines()
target_lines = target.readlines()

In [8]:
source_lines[0:10]

['태초 에 하나님 이 천지 를 창조 하 시 었 다 .\n',
 '땅 이 혼돈 하 고 공허 하 며 , 어둠 이 깊 음 위 에 있 고 , 하 나 님 의 영 은 물 위 에 움직이 고 계시 었 다 .\n',
 '하나님 이 말씀 하 시 기 를 " 빛 이 생기 어라 " 하 시 니 , 빛 이 생기 었 다 .\n',
 '그 빛 이 하나님 보 시 기 에 좋 았 다 . 하나님 이 빛 과 어둠 을 나누 시 어서 ,\n',
 '빛 을 낮 이 라고 하 시 고 , 어둠 을 밤 이 라고 하 시 었 다 . 저녁 이 되 고 아침 이 되 니 , 하루 가 지나 았 다 .\n',
 '하나님 이 말씀 하 시 기 를 " 물 한 가운데 창공 이 생기 어 , 물 과 물 사이 가 갈 아 지 어라 " 하 시 었 다 .\n',
 '하나님 이 이 처럼 창공 을 만들 시 고서 , 물 을 창공 아래 에 있 는 물 과 창공 위 에 있 는 물 로 나누 시 니 , 그대로 되 었 다 .\n',
 '하나님 이 창공 을 하늘 이 라고 하 시 었 다 . 저녁 이 되 고 아침 이 되 니 , 이튿날 이 지나 았 다 .\n',
 '하나님 이 말씀 하 시 기 를 " 하늘 아래 에 있 는 물 은 한 곳 으로 모이 고 , 뭍 은 드러나 거라 " 하 시 니 , 그대로 되 었 다 .\n',
 '하나님 이 뭍 을 땅 이 라고 하 시 고 , 모이 ㄴ 물 을 바다 이 라고 하 시 었 다 . 하나님 보 시 기 에 좋 았 다 .\n']

In [6]:
target_lines[0:10]

['태초 에 하나님 이 천지 를 창조 하 시 니라\n',
 '땅 이 혼돈 하 고 공허 하 며 흑 암 이 깊 음 위 에 있 고 하나님 의 영 은 수면 위 에 운행 하 시 니라\n',
 '하나님 이 이 르시 되 빛 이 있 으라 하 시 니 빛 이 있 었 고\n',
 '빛 이 하나님 이 보 시 기 에 좋 았 더라 하나님 이 빛 과 어둠 을 나누 사\n',
 '하나님 이 빛 을 낮 이 라 부르 시 고 어둠 을 밤 이 라 부르 시 니라 저녁 이 되 고 아침 이 되 니 이 는 첫째 날 이 니라\n',
 '하나님 이 이 르시 되 물 가운데 에 궁창 이 있 어 물 과 물 로 나뉘 라 하 시 고\n',
 '하나님 이 궁창 을 만들 사 궁창 아래 의 물 과 궁창 위 의 물 로 나뉘 게 하 시 니 그대로 되 니라\n',
 '하나님 이 궁창 을 하늘 이 라 부르 시 니라 저녁 이 되 고 아침 이 되 니 이 는 둘째 날 이 니라\n',
 '하나님 이 이 르시 되 천하 의 물 이 한 곳 으로 모이 고 뭍 이 드러나 라 하 시 니 그대로 되 니라\n',
 '하나님 이 뭍 을 땅 이 라 부르 시 고 모이 ㄴ 물 을 바다 라 부르 시 니 하나님 이 보 시 기 에 좋 았 더라\n']

In [9]:
tokenizer_source = tfds.features.text.SubwordTextEncoder.build_from_corpus((line for line in source_lines), target_vocab_size=10000)
tokenizer_target = tfds.features.text.SubwordTextEncoder.build_from_corpus((line for line in target_lines), target_vocab_size=10000)

In [10]:
text = '이순신은 조선 중기의 무신입니다.'

tokenized_string = tokenizer_source.encode(text)
print(tokenized_string)
for ts in tokenized_string:
    print ('{}: {}'.format(tokenizer_source.decode([ts]), ts))

[239, 2791, 1957, 13, 2060, 858, 3276, 1172, 5, 1701, 1957, 3282, 1606, 9296]
이: 239
순: 2791
신: 1957
은 : 13
조: 2060
선 : 858
중: 3276
기: 1172
의 : 5
무: 1701
신: 1957
입: 3282
니다: 1606
.: 9296


In [11]:
# save the vocabs
tokenizer_source.save_to_file('/content/seq2seq_source_vocab')
tokenizer_target.save_to_file('/content/seq2seq_target_vocab')


# Load
tokenizer_source = tfds.features.text.SubwordTextEncoder.load_from_file('/content/seq2seq_source_vocab')
tokenizer_target= tfds.features.text.SubwordTextEncoder.load_from_file('/content/seq2seq_target_vocab')

In [12]:
BUFFER_SIZE = 30000
BATCH_SIZE = 64
MAX_LENGTH = 128

In [13]:
def encode(lang1, lang2):
    lang1 = [tokenizer_source.vocab_size] + tokenizer_source.encode(lang1.numpy()) + [tokenizer_source.vocab_size+1]
    lang2 = [tokenizer_target.vocab_size] + tokenizer_target.encode(lang2.numpy()) + [tokenizer_target.vocab_size+1]
    
    return lang1, lang2

In [14]:
def tf_encode(src, tar):
    return tf.py_function(encode, [src, tar], [tf.int64, tf.int64])

In [15]:
src_dataset = tf.data.TextLineDataset('/content/src')
target_dataset = tf.data.TextLineDataset('/content/target')
lines_dataset = tf.data.Dataset.zip((src_dataset, target_dataset))

for ex in lines_dataset.take(5):
    print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b'\xed\x83\x9c\xec\xb4\x88 \xec\x97\x90 \xed\x95\x98\xeb\x82\x98\xeb\x8b\x98 \xec\x9d\xb4 \xec\xb2\x9c\xec\xa7\x80 \xeb\xa5\xbc \xec\xb0\xbd\xec\xa1\xb0 \xed\x95\x98 \xec\x8b\x9c \xec\x97\x88 \xeb\x8b\xa4 .'>, <tf.Tensor: shape=(), dtype=string, numpy=b'\xed\x83\x9c\xec\xb4\x88 \xec\x97\x90 \xed\x95\x98\xeb\x82\x98\xeb\x8b\x98 \xec\x9d\xb4 \xec\xb2\x9c\xec\xa7\x80 \xeb\xa5\xbc \xec\xb0\xbd\xec\xa1\xb0 \xed\x95\x98 \xec\x8b\x9c \xeb\x8b\x88\xeb\x9d\xbc'>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'\xeb\x95\x85 \xec\x9d\xb4 \xed\x98\xbc\xeb\x8f\x88 \xed\x95\x98 \xea\xb3\xa0 \xea\xb3\xb5\xed\x97\x88 \xed\x95\x98 \xeb\xa9\xb0 , \xec\x96\xb4\xeb\x91\xa0 \xec\x9d\xb4 \xea\xb9\x8a \xec\x9d\x8c \xec\x9c\x84 \xec\x97\x90 \xec\x9e\x88 \xea\xb3\xa0 , \xed\x95\x98 \xeb\x82\x98 \xeb\x8b\x98 \xec\x9d\x98 \xec\x98\x81 \xec\x9d\x80 \xeb\xac\xbc \xec\x9c\x84 \xec\x97\x90 \xec\x9b\x80\xec\xa7\x81\xec\x9d\xb4 \xea\xb3\xa0 \xea\xb3\x84\xec\x8b\x9c \xec\x97\x88 

In [16]:
train_dataset = lines_dataset.map(tf_encode)
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([-1], [-1]))
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [17]:
en_batch, kr_batch = next(iter(train_dataset))
print(en_batch, kr_batch)

tf.Tensor(
[[9504    8   11 ...    0    0    0]
 [9504  369    1 ...    0    0    0]
 [9504  188   21 ...    0    0    0]
 ...
 [9504    8   11 ...    0    0    0]
 [9504   54    5 ...    0    0    0]
 [9504    8 1568 ...    0    0    0]], shape=(64, 88), dtype=int64) tf.Tensor(
[[10609     5     3 ...     0     0     0]
 [10609   344     1 ...     0     0     0]
 [10609   144    29 ...     0     0     0]
 ...
 [10609     5     6 ...     0     0     0]
 [10609    51   215 ...     0     0     0]
 [10609     5    10 ...     0     0     0]], shape=(64, 82), dtype=int64)


Model

In [18]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis], np.arange(d_model)[np.newaxis, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    
    pos_encoding = angle_rads[np.newaxis, ...]
    
    return tf.cast(pos_encoding, dtype=tf.float32)

In [19]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

In [20]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

In [21]:
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    output = tf.matmul(attention_weights, v)

    return output, attention_weights

In [22]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
    
        assert d_model % self.num_heads == 0
    
        self.depth = d_model // self.num_heads
    
        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)
    
        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]
    
        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)
    
        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)
    
        scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)
    
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))

        output = self.dense(concat_attention)
        
        return output, attention_weights

In [23]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([tf.keras.layers.Dense(dff, activation='relu'), tf.keras.layers.Dense(d_model)])

In [24]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
    
    def call(self, x, training, mask):

        attn_output, _ = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)
    
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
    
        return out2

In [25]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)
        
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):

        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)
    
        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)
    
        ffn_output = self.ffn(out2) 
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)
    
        return out3, attn_weights_block1, attn_weights_block2

In [26]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers
    
        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        
        self.dropout = tf.keras.layers.Dropout(rate)
        
    def call(self, x, training, mask):

        seq_len = tf.shape(x)[1]
    
        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)
    
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
    
        return x

In [27]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers
    
        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
    
        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):

        seq_len = tf.shape(x)[1]
        attention_weights = {}
    
        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]
    
        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training,look_ahead_mask, padding_mask)
            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
        return x, attention_weights

In [28]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()
        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)
        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)
        self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
    def call(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):

        enc_output = self.encoder(inp, training, enc_padding_mask)
        dec_output, attention_weights = self.decoder(tar, enc_output, training, look_ahead_mask, dec_padding_mask)
    
        final_output = self.final_layer(dec_output)
    
        return final_output, attention_weights

In [29]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8

input_vocab_size = tokenizer_source.vocab_size + 2
target_vocab_size = tokenizer_target.vocab_size + 2
dropout_rate = 0.1

In [30]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
    
        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
    
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [31]:
learning_rate = CustomSchedule(d_model)
# learning_rate = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)
# optimizer = tf.keras.optimizers.Adam(learning_rate)

In [32]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [33]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_mean(loss_)

In [34]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

In [35]:
transformer = Transformer(num_layers, d_model, num_heads, dff,
                          input_vocab_size, target_vocab_size, 
                          pe_input=input_vocab_size, 
                          pe_target=target_vocab_size,
                          rate=dropout_rate)

In [36]:
def create_masks(inp, tar):
    enc_padding_mask = create_padding_mask(inp)
    dec_padding_mask = create_padding_mask(inp)

    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
    
    return enc_padding_mask, combined_mask, dec_padding_mask

In [37]:
checkpoint_path = '/content/bible_translator'

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint is restored')

In [38]:
EPOCHS = 5

In [39]:
train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]

@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]
    
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)
    
    with tf.GradientTape() as tape:
        predictions, _ = transformer(inp, tar_inp, True, enc_padding_mask, combined_mask, dec_padding_mask)
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
    train_loss(loss)
    train_accuracy(tar_real, predictions)

In [40]:
for epoch in range(EPOCHS):
    start = time.time()
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    for (batch, (inp, tar)) in enumerate(train_dataset):
        train_step(inp, tar)
    
        if batch % 50 == 0:
            print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, batch, train_loss.result(), train_accuracy.result()))
            
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))
    
    print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, train_loss.result(), train_accuracy.result()))

    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.1601 Accuracy 0.0000
Epoch 1 Batch 50 Loss 4.2216 Accuracy 0.0024
Epoch 1 Batch 100 Loss 4.1677 Accuracy 0.0084
Epoch 1 Batch 150 Loss 4.1057 Accuracy 0.0117
Epoch 1 Batch 200 Loss 4.0129 Accuracy 0.0139
Epoch 1 Batch 250 Loss 3.9110 Accuracy 0.0154
Epoch 1 Batch 300 Loss 3.7999 Accuracy 0.0166
Epoch 1 Batch 350 Loss 3.6821 Accuracy 0.0177
Epoch 1 Batch 400 Loss 3.5768 Accuracy 0.0193
Epoch 1 Batch 450 Loss 3.4733 Accuracy 0.0212
Epoch 1 Loss 3.4148 Accuracy 0.0229
Time taken for 1 epoch: 81.58805179595947 secs

Epoch 2 Batch 0 Loss 2.4411 Accuracy 0.0487
Epoch 2 Batch 50 Loss 2.5418 Accuracy 0.0520
Epoch 2 Batch 100 Loss 2.5123 Accuracy 0.0586
Epoch 2 Batch 150 Loss 2.4649 Accuracy 0.0653
Epoch 2 Batch 200 Loss 2.3952 Accuracy 0.0704
Epoch 2 Batch 250 Loss 2.3559 Accuracy 0.0757
Epoch 2 Batch 300 Loss 2.3111 Accuracy 0.0797
Epoch 2 Batch 350 Loss 2.2733 Accuracy 0.0832
Epoch 2 Batch 400 Loss 2.2336 Accuracy 0.0859
Epoch 2 Batch 450 Loss 2.2118 Accuracy 0.0892
Ep

KeyboardInterrupt: ignored

In [41]:
def evaluate(inp_sentence):
    start_token = [tokenizer_source.vocab_size]
    end_token = [tokenizer_source.vocab_size + 1]
    
    inp_sentence = start_token + tokenizer_source.encode(inp_sentence) + end_token
    encoder_input = tf.expand_dims(inp_sentence, 0)

    decoder_input = [tokenizer_target.vocab_size]
    output = tf.expand_dims(decoder_input, 0)
    
    for i in range(MAX_LENGTH):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)
        predictions, attention_weights = transformer(encoder_input, output, False, enc_padding_mask, combined_mask, dec_padding_mask)
    
        predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
        
        if predicted_id >= tokenizer_target.vocab_size+1:
            return tf.squeeze(output, axis=0), attention_weights
        
        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights

In [44]:
def prediction(sentence):
    result, attention_weights = evaluate(sentence)
    predicted_sentence = tokenizer_target.decode([i for i in result 
                                            if i < tokenizer_target.vocab_size])  
    print('Bible: {}'.format(predicted_sentence))

In [45]:
prediction('집 에 가 고 싶 다 .')

Bible: 네 가 집 에 들어가 아서 죽 은 것 을 보 고 마시 니
